###Select the business type of your choosing

In [ ]:
options = {'accounting','airport','amusement_park','aquarium','art_gallery','atm','bakery','bank','bar','beauty_salon','bicycle_store','book_store','bowling_alley','bus_station','cafe','campground','car_dealer','car_rental','car_repair','car_wash','casino','cemetery','church','city_hall','clothing_store','convenience_store','courthouse','dentist','department_store','doctor','drugstore','electrician','electronics_store','embassy','fire_station','florist','funeral_home','furniture_store','gas_station','gym','hair_care','hardware_store','hindu_temple','home_goods_store','hospital','insurance_agency','jewelry_store','laundry','lawyer','library','light_rail_station','liquor_store','local_government_office','locksmith','lodging','meal_delivery','meal_takeaway','mosque','movie_rental','movie_theater','moving_company','museum','night_club','painter','park','parking','pet_store','pharmacy','physiotherapist','plumber','police','post_office','primary_school','real_estate_agency','restaurant','roofing_contractor','rv_park','school','secondary_school','shoe_store','shopping_mall','spa','stadium','storage','store','subway_station','supermarket','synagogue','taxi_stand','tourist_attraction','train_station','transit_station','travel_agency','university','veterinary_care','zoo'}
print('The options are: ')
cnt = 0
for key in options:
  print(key + ' '*(25 - len(key)), end='\t')
  cnt+=1
  if cnt==6:
    cnt = 0
    print('\n')


The options are: 
book_store               	department_store         	police                   	hardware_store           	clothing_store           	home_goods_store         	

meal_takeaway            	synagogue                	light_rail_station       	local_government_office  	transit_station          	beauty_salon             	

supermarket              	gym                      	car_wash                 	drugstore                	bank                     	funeral_home             	

train_station            	cafe                     	bus_station              	lawyer                   	roofing_contractor       	stadium                  	

gas_station              	real_estate_agency       	fire_station             	store                    	moving_company           	university               	

church                   	lodging                  	city_hall                	taxi_stand               	aquarium                 	atm                      	

pet_store                	locksmit

In [ ]:
business_type = 'shopping_mall'

#####.

In [ ]:
pip install googlemaps

In [ ]:
import googlemaps
from datetime import datetime
import pandas as pd
from math import pi, cos
import time

In [ ]:
gmaps = googlemaps.Client(key='#####')

In [ ]:
# geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')
# print(geocode_result)

## Dividing Addis Ababa into grids

In [ ]:
#Bottom Left corner of Addis Ababa
bottom_left_corner = [8.834616884187174, 38.65314470981974]
#top right corner of addis ababa
top_right_corner = [9.098837418615586, 38.91274583251382]

r_earth = 6378137
dy = dx = 1000

coordinates = []
while bottom_left_corner[0]<=top_right_corner[0]:
    x = bottom_left_corner[1]
    while x<=top_right_corner[1]:
        x = x + (dx / r_earth) * (180 / pi) / cos(bottom_left_corner[0] * pi/180);
        coordinates.append((bottom_left_corner[0], x))
    bottom_left_corner[0] +=  + (dy / r_earth) * (180 / pi)

print(coordinates[:5], '......', coordinates[-5:])
print(len(coordinates))

[(8.834616884187174, 38.66223572049971), (8.834616884187174, 38.671326731179676), (8.834616884187174, 38.68041774185964), (8.834616884187174, 38.68950875253961), (8.834616884187174, 38.698599763219576)] ...... [(9.095128316581816, 38.8805830561192), (9.095128316581816, 38.88968058997118), (9.095128316581816, 38.898778123823156), (9.095128316581816, 38.907875657675135), (9.095128316581816, 38.91697319152711)]
870


## For each grid, based on a 1km radius find the business type of your choosing

In [ ]:
data_list = []
#Collect every business of your choosing into a list by calling google's api
for coordinate in coordinates:
    params = {
        'location':coordinate,
        'radius':1000,
        'type': business_type
    }

    first_page = gmaps.places_nearby(**params)
    
    second_page = {'results':[]}
    third_page = {'results':[]}
    
    #Fetching the second page if there is any
    if 'next_page_token' in first_page:
        params['page_token'] = first_page['next_page_token']
        time.sleep(2)
        second_page = gmaps.places_nearby(**params)
    
    #Fetching the third or last page if there is any
    if 'next_page_token' in second_page:
        params['page_token'] = second_page['next_page_token']
        time.sleep(2)
        third_page = gmaps.places_nearby(**params)
    
    data_list.extend(first_page['results'])
    data_list.extend(second_page['results'])
    data_list.extend(third_page['results'])

In [ ]:
df = pd.DataFrame(data_list)    
df.head()

,business_status,geometry,icon,name,opening_hours,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,permanently_closed
0,OPERATIONAL,"{'location': {'lat': 8.83577, 'lng': 38.844928...",https://maps.gstatic.com/mapfiles/place_api/ic...,Ato Minesinot,{'open_now': False},"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJlev9IbV3SxYRgMGx1Jv1ANc,"{'compound_code': 'RRPV+8X Akaki, Ethiopia', '...",4.5,ChIJlev9IbV3SxYRgMGx1Jv1ANc,GOOGLE,"[shopping_mall, point_of_interest, establishment]",2.0,Ethiopia,NaN
1,OPERATIONAL,"{'location': {'lat': 8.83577, 'lng': 38.844928...",https://maps.gstatic.com/mapfiles/place_api/ic...,Ato Minesinot,{'open_now': False},"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJlev9IbV3SxYRgMGx1Jv1ANc,"{'compound_code': 'RRPV+8X Akaki, Ethiopia', '...",4.5,ChIJlev9IbV3SxYRgMGx1Jv1ANc,GOOGLE,"[shopping_mall, point_of_interest, establishment]",2.0,Ethiopia,NaN
2,OPERATIONAL,"{'location': {'lat': 8.83577, 'lng': 38.844928...",https://maps.gstatic.com/mapfiles/place_api/ic...,Ato Minesinot,{'open_now': False},"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJlev9IbV3SxYRgMGx1Jv1ANc,"{'compound_code': 'RRPV+8X Akaki, Ethiopia', '...",4.5,ChIJlev9IbV3SxYRgMGx1Jv1ANc,GOOGLE,"[shopping_mall, point_of_interest, establishment]",2.0,Ethiopia,NaN
3,OPERATIONAL,"{'location': {'lat': 8.8666158, 'lng': 38.7859...",https://maps.gstatic.com/mapfiles/place_api/ic...,Doro Tara,{'open_now': False},NaN,ChIJ-TVT-Ph5SxYRn7FKsNHA_DU,"{'compound_code': 'VQ8P+J9 Addis Ababa, Ethiop...",NaN,ChIJ-TVT-Ph5SxYRn7FKsNHA_DU,GOOGLE,"[shopping_mall, point_of_interest, establishment]",NaN,5/6,NaN
4,OPERATIONAL,"{'location': {'lat': 8.8666158, 'lng': 38.7859...",https://maps.gstatic.com/mapfiles/place_api/ic...,Doro Tara,{'open_now': False},NaN,ChIJ-TVT-Ph5SxYRn7FKsNHA_DU,"{'compound_code': 'VQ8P+J9 Addis Ababa, Ethiop...",NaN,ChIJ-TVT-Ph5SxYRn7FKsNHA_DU,GOOGLE,"[shopping_mall, point_of_interest, establishment]",NaN,5/6,NaN


In [ ]:
print("Length of rows and columns", df.shape)
print("Columns", df.columns)

Length of rows and columns (2555, 15)
Columns Index(['business_status', 'geometry', 'icon', 'name', 'opening_hours',
       'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope',
       'types', 'user_ratings_total', 'vicinity', 'permanently_closed'],
      dtype='object')


## Cleaning the dataset by extracting coordinates and dividing the types into separate columns

In [ ]:
df.columns

Index(['business_status', 'geometry', 'icon', 'name', 'opening_hours',
       'photos', 'place_id', 'plus_code', 'rating', 'reference', 'scope',
       'types', 'user_ratings_total', 'vicinity', 'permanently_closed'],
      dtype='object')

In [ ]:
def lat(row):
    if 'location' in row:
        return row['location']['lat']
    return None
def lng(row):
    if 'location' in row:
        return row['location']['lng']
    return None
def check(row, col):
    if col in row:
        return 1
    return 0

#Remove unnecessary columns
cleaned_df = df.drop(['photos', 'icon', 'scope', 'permanently_closed', 'opening_hours', 'reference','plus_code', 'user_ratings_total', 'vicinity'], axis=1)

# Extract Location Information
cleaned_df['latitude'] = cleaned_df['geometry'].apply(lat)
cleaned_df['longitude'] = cleaned_df['geometry'].apply(lng)
# print(cleaned_df.columns)

#Remove unnecessary columns after cleaning up
cleaned_df = cleaned_df.drop(['geometry', 'types'],axis=1)

#Drop duplicates
print('Before dropping duplicates, the size was,',len(cleaned_df))
cleaned_df.sort_values("place_id", inplace=True)
cleaned_df.drop_duplicates(inplace=True)
print('After dropping duplicates, the size was,',len(cleaned_df))


Before dropping duplicates, the size was, 2555
After dropping duplicates, the size was, 804


In [ ]:
cleaned_df.head()

,business_status,name,place_id,rating,permanently_closed,latitude,longitude,moving_company,aquarium,car_dealer,city_hall,cemetery,store,art_gallery,electrician,embassy,home_goods_store,university,real_estate_agency,supermarket,bar,local_government_office,transit_station,fire_station,secondary_school,primary_school,subway_station,car_repair,lawyer,car_rental,furniture_store,storage,roofing_contractor,travel_agency,painter,school,book_store,museum,restaurant,rv_park,...,pharmacy,plumber,movie_rental,police,parking,physiotherapist,courthouse,accounting,doctor,bowling_alley,park,church,zoo,amusement_park,night_club,hindu_temple,clothing_store,light_rail_station,hair_care,mosque,shoe_store,bus_station,shopping_mall,insurance_agency,bank,florist,spa,cafe,convenience_store,synagogue,casino,locksmith,tourist_attraction,train_station,funeral_home,lodging,gym,hospital,gas_station,post_office
800,OPERATIONAL,Aledolina Mobile አልዶሊና ሞባይል,ChIJ--asYZqFSxYRmwwbek8fUTY,NaN,NaN,8.997097,38.766705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
264,OPERATIONAL,I TECH,ChIJ--wPcnKHSxYRqs4f4OTS9cE,NaN,NaN,8.973212,38.681260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1451,OPERATIONAL,Smart Plaza,ChIJ-3EAGpyFSxYRJ2G0WrsnvOs,4.8,NaN,9.016115,38.738668,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
712,OPERATIONAL,Hilina Building ሕሊና ሕንፃ,ChIJ-4k1s3CbSxYR5rt9ewqtg58,3.5,NaN,8.990858,38.829367,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2407,OPERATIONAL,Amenu Building,ChIJ-7L01u-JSxYRWQ1szbNkpgc,3.0,NaN,9.046220,38.732703,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Export data set

In [ ]:
cleaned_df.to_csv('../Data/' + business_type + '.csv')